In [1]:
import glob as glob
import os

import cv2
import numpy as np
import pandas as pd
from IPython.display import clear_output
from sklearn.metrics import (
    classification_report,
    cohen_kappa_score,
    confusion_matrix,
    roc_auc_score,
)

In [2]:
# Load the labels
LABELS = open("/Users/tasanders/Google Drive/Square Eyes (DP20)/5 - Data collection and management/Image Blurring Test/Yolo/coco.names").read().strip().split("\n")

In [3]:
# Get the weights and config
configpath = "/Users/tasanders/Google Drive/Square Eyes (DP20)/5 - Data collection and management/Image Blurring Test/Yolo/yolov4.cfg"
weightspath = "/Users/tasanders/Google Drive/Square Eyes (DP20)/5 - Data collection and management/Image Blurring Test/Yolo/yolov4.weights"

net = cv2.dnn.readNet(configpath, weightspath)


# Determine the output layer names
ln = net.getLayerNames()
ln = [ln[i[0]-1] for i in net.getUnconnectedOutLayers()]

In [4]:
# Function to make a prediction and save to df
def predict_and_save(folder, coded_data, conf_thresh=0.25, nms_thresh=0.2):
    predict_files = glob.glob(folder + "/*/*.jpg")

    predictor, prob, image_id = [], [], []
    final = pd.DataFrame(columns=["id", "prediction", "confidence", "image"])

    not_coded = []

    for index, image in enumerate(predict_files):
        if coded_data["filename"].str.contains(os.path.basename(image)).any():
            clear_output(wait=True)
            print(f"Working on image {index} of {len(predict_files)-1}")

            im = cv2.imread(image)
            (H, W) = im.shape[:2]

            # Create the blob
            blob = cv2.dnn.blobFromImage(
                im, 1 / 255.0, (416, 416), swapRB=True, crop=False
            )
            net.setInput(blob)
            layerOutputs = net.forward(ln)

            # Translate the predictions
            boxes = []
            confidences = []
            classIDs = []

            for output in layerOutputs:
                for detection in output:
                    scores = detection[5:]
                    classID = np.argmax(scores)
                    confidence = scores[classID]

                    if confidence > conf_thresh:
                        box = detection[0:4] * np.array([W, H, W, H])
                        (centerX, centerY, width, height) = box.astype("int")
                        x = int(centerX - (width / 2))
                        y = int(centerY - (height / 2))
                        # update our list of bounding box coordinates, confidences, and class IDs
                        boxes.append([x, y, int(width), int(height)])
                        confidences.append(float(confidence))
                        classIDs.append(classID)

            # apply non-maxima suppression to suppress weak, overlapping bounding boxes
            idxs = cv2.dnn.NMSBoxes(boxes, confidences, conf_thresh, nms_thresh)

            # Append to df
            if len(idxs):
                for i in idxs.flatten():
                    final = final.append(
                        {
                            "id": classIDs[i],
                            "prediction": LABELS[classIDs[i]],
                            "confidence": confidences[i],
                            "image": os.path.basename(image),
                        },
                        ignore_index=True,
                    )
            else:  # no predictions made
                final = final.append(
                    {
                        "id": None,
                        "prediction": None,
                        "confidence": None,
                        "image": os.path.basename(image),
                    },
                    ignore_index=True,
                )

    return final

##  Test against Bridget's coding

In [5]:
%%time
coded_data = pd.read_csv("/Volumes/M&B/Screen_Time_Measure_Development/SNAP_IT/Coding Framework Test Images/Screen Time Coding Data - Device.csv")
folder = "/Volumes/M&B/Screen_Time_Measure_Development/SNAP_IT/Coding Framework Test Images"
df = predict_and_save(folder, coded_data)

Working on image 4495 of 4495
CPU times: user 3h 21min 31s, sys: 4min 16s, total: 3h 25min 47s
Wall time: 40min 43s


In [6]:
df_backup = df.copy()

In [7]:
cat_maybe = [
    "mouse",
    "remote",
    "keyboard",
]

cat_def = [
    "tvmonitor",
    "laptop",
    "cell phone",
]

In [8]:
df['screen_def'] = np.where(df["prediction"].isin(cat_def),1,0)
df['screen_maybe'] = np.where(df["prediction"].isin(cat_def + cat_maybe),1,0)

In [9]:
def conf_def(df, confthresh):
    if df["confidence"] > confthresh and df["screen_def"]==1:
        return 1
    else:
        return 0
def conf_maybe(df, confthresh):
    if df["confidence"] > confthresh and df["screen_maybe"]==1:
        return 1
    else:
        return 0

In [10]:
for x in np.arange(0.2, 0.8,0.05):
    df["screen_def_"+str(x)] = df.apply(conf_def, confthresh=x, axis=1)
    df["screen_maybe_"+str(x)] = df.apply(conf_maybe, confthresh=x, axis=1)

In [11]:
df = df.merge(coded_data,left_on="image", right_on="filename")
df.drop(columns="filename", inplace=True)
df[["device","device_excl_bkg"]] = df[["device","device_excl_bkg"]].astype(int)
df = df.drop(columns=["prediction","confidence","id"]).groupby(["image"]).any()

In [12]:
df.columns

Index(['screen_def', 'screen_maybe', 'screen_def_0.2', 'screen_maybe_0.2',
       'screen_def_0.25', 'screen_maybe_0.25', 'screen_def_0.3',
       'screen_maybe_0.3', 'screen_def_0.35', 'screen_maybe_0.35',
       'screen_def_0.39999999999999997', 'screen_maybe_0.39999999999999997',
       'screen_def_0.44999999999999996', 'screen_maybe_0.44999999999999996',
       'screen_def_0.49999999999999994', 'screen_maybe_0.49999999999999994',
       'screen_def_0.5499999999999999', 'screen_maybe_0.5499999999999999',
       'screen_def_0.5999999999999999', 'screen_maybe_0.5999999999999999',
       'screen_def_0.6499999999999999', 'screen_maybe_0.6499999999999999',
       'screen_def_0.7', 'screen_maybe_0.7', 'screen_def_0.7499999999999998',
       'screen_maybe_0.7499999999999998', 'screen_def_0.7999999999999998',
       'screen_maybe_0.7999999999999998', 'device', 'device_excl_bkg'],
      dtype='object')

In [13]:
true_devices = ['device', 'device_excl_bkg']
predicted_devices = df.drop(columns=['device', 'device_excl_bkg','screen_def', 'screen_maybe',])


In [16]:
rocs = []
for true_device in true_devices:
    for predicted_device in predicted_devices:
        print(f"Comparing: {true_device} & {predicted_device}")
        print(classification_report(df[true_device], df[predicted_device]))
        print(roc_auc_score(df[true_device], df[predicted_device]))
        rocs.append(roc_auc_score(df[true_device], df[predicted_device]))

Comparing: device & screen_def_0.2
              precision    recall  f1-score   support

       False       0.68      0.93      0.79      1484
        True       0.96      0.79      0.86      3011

    accuracy                           0.83      4495
   macro avg       0.82      0.86      0.83      4495
weighted avg       0.87      0.83      0.84      4495

0.8595321198731337
Comparing: device & screen_maybe_0.2
              precision    recall  f1-score   support

       False       0.69      0.93      0.79      1484
        True       0.96      0.79      0.87      3011

    accuracy                           0.84      4495
   macro avg       0.82      0.86      0.83      4495
weighted avg       0.87      0.84      0.84      4495

0.8619989284572919
Comparing: device & screen_def_0.25
              precision    recall  f1-score   support

       False       0.68      0.93      0.79      1484
        True       0.96      0.79      0.86      3011

    accuracy                        

In [21]:
rocs

[0.8619989284572919,
 0.8619989284572919,
 0.8595321198731337,
 0.8595321198731337,
 0.8561026460928081,
 0.8531280408493206,
 0.8488682781284437,
 0.8443895294969658,
 0.8418755191669135,
 0.8418755191669135,
 0.839675926711581,
 0.839675926711581,
 0.8372923494357168,
 0.8372570277087731,
 0.8341038433907577,
 0.8323154274399082,
 0.8293624271317009,
 0.8280316064815354,
 0.8247179365342517,
 0.8222243955451752,
 0.8181186152532055,
 0.8155965189632622,
 0.8129607776331689,
 0.8102971168586253,
 0.8092911346625714,
 0.8042837008676097,
 0.8008366224114455,
 0.7988329106025578,
 0.7941991225345342,
 0.7923061475889552,
 0.7854620645361732,
 0.783244120166756,
 0.778593070174179,
 0.7757763313492934,
 0.7701592364385393,
 0.7692309867967886,
 0.762857505409187,
 0.7615064140638155,
 0.7574981952898697,
 0.7542369353699507,
 0.7499447382310925,
 0.7469352043405983,
 0.7419515802472681,
 0.734398123188491,
 0.733166842869944,
 0.7260263579812027,
 0.7225673181898681,
 0.7152093441994506,

In [19]:
print(f"Comparing: device & screen_maybe_0.3")
print(classification_report(df["device"], df["screen_maybe_0.3"]))
print(cohen_kappa_score(df["device"], df["screen_maybe_0.3"]))
print(f"Comparing: device_excl_bkg & screen_maybe_0.3")
print(classification_report(df["device_excl_bkg"], df["screen_maybe_0.3"]))
print(cohen_kappa_score(df["device_excl_bkg"], df["screen_maybe_0.3"]))

Comparing: device & screen_maybe_0.3
              precision    recall  f1-score   support

       False       0.67      1.00      0.80        10
        True       1.00      0.83      0.91        29

    accuracy                           0.87        39
   macro avg       0.83      0.91      0.85        39
weighted avg       0.91      0.87      0.88        39

0.711111111111111
Comparing: device_excl_bkg & screen_maybe_0.3
              precision    recall  f1-score   support

       False       0.67      1.00      0.80        10
        True       1.00      0.83      0.91        29

    accuracy                           0.87        39
   macro avg       0.83      0.91      0.85        39
weighted avg       0.91      0.87      0.88        39

0.711111111111111


In [20]:

print(cohen_kappa_score(df["device_excl_bkg"], df["screen_maybe_0.3"]))

0.711111111111111


In [21]:
print(roc_auc_score(df["device"], df["screen_maybe_0.3"]))

0.9137931034482758
